# 06-Extension to the WCPS core: XPath-Enabled WCPS (xWCPS)

<a href="./05-WCPS_intro.ipynb"><< 05-WCPS_extension</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./07-Service_overview.ipynb">07-Intro to Services>></a>

## Why xWCPS?

xWCPS created to offer additional features to the WCPS service described in the [previous tutorial](./03-WCPS_extension.ipynb).
These features consist of:

* ### Coverage Identification Based on Metadata
One of the fundamental operations a query language must offer is that of querying for all data residing in the database without prior knowledge of their internal representation. WCPS requires the specification of coverage identifiers in selection queries. 
xWCPS is introduced to fill this gap offering a unified language by combining two already well known standards, WCPS and XPath 2.0, aiming at being rich, expressive and user friendly. 
In the following query all coverages from the "access.planetserver.eu" service will be encoded in 'png' format and returned as a result.

>**Example**: *<br /> for \$c in /server[@endpoint='http://access.planetserver.eu:8080/rasdaman/ows']/coverage <br />
          return encode(\$c, "png")*

* ### Exploitation of Descriptive Metadata
Coverage filtering based on the available metadata using XPath 2.0. For and where clauses can contain XPath 2.0 expressions in order to restrict results to specific metadata.

>**Example**: *<br /> for \$c in /server/coverage <br /> metadata(\$c)\$c//gml:cat_solar_longitude[text()>86.122] <br /> return encode(\$c, "png")*

* ### Repetitiveness Reduction
xWCPS supports variable manipulation via 'let' clause, which allows assigning complex expressions to variables and using them for subsequent references, avoiding repetitiveness.

>**Example**: *<br /> let \$value = 85; <br /> for \$c in /server/coverage <br /> where metadata(\$c)\$c//gml:cat_solar_longitude[text()>\$value] <br /> return encode(\$c, "png")*

* ### Extended Set of Results Support
An important feature of xWCPS is the ability to return the data accompanied with their metadata. This can be achieved using the 'mixed' clause of xWCPS.

>**Example**: *<br /> for \$c in /server/coverage <br /> return mixed(encode(\$c, "png", metadata(\$c))*

## XPath-Enabled Web Coverage Processing Service (xWCPS) requests

A xWCPS quer consists of three components:

* **For** variableName **in** <font color="green">for expression</font>,
* **where** <font color="red">where expression</font> (optional)
* **return** <font color="blue">return expression</font>.

Moreover may contain variable declaration (via **let** clause) anywhere in a query.   


 
> <font color="green">For expression</font> may contain:  
* List of coverage ids
* XPath 2.0 expression (ex. specific endpoints)

>> Examples: 
+ for coverage in (coverageId1, coverageId2, ... )
+ for coverage in /server/coverage
+ for coverage in /server[@endpoint='http://access.planetserver.eu:8080/rasdaman/ows']/coverage

> <font color="red">Where expression</font> may contain:  
* Metadata expression
* XPath 2.0 expression

>> Example: where metadata(c)//gml:cat_solar_longitude[text()<86.0122]

> <font color="blue">Return expression</font> may contain:  
* Metadata expression
* XPath 2.0 expression
* Mixed function

>> Examples: 
* return metadata(c)
* return metadata(c)//gml:cat_solar_longitude
* return mixed(encode (return {red: 0; green: 0; blue: 128} ,"png"), metadata(c))


***

## <a id=xwcps_examples></a>Examples: xWCPS requests

The following examples show a variety of xWCPS queries that can be applied to the data of a WCS Server, when metadata are harvested and stored in database. Examples are based on the following services: 
http://access.planetserver.eu:8080/rasdaman/ows and https://rsg.pml.ac.uk/rasdaman/ows

- [Basic xWCPS request](#basic_xwcps)
- [xWCPS request with Xpath on For/Where Clause](#xpath_for_where_xwcps)
- [xWCPS request with Xpath on Return Clause](#xpath_return_xwcps)
- [xWCPS request with Mixed Result](#xpath_mixed_result)

All requests can be executed in http://earthserver-devel.vhosts.cite.gr/

### <a id='basic_xwcps'></a><font color="black">Basic xWCPS requests</font>

* Retrieve metadata from a specific coverage with id **CCI_V2_monthly_rrs_670** 

>for \$c in ( CCI_V2_monthly_rrs_670 ) return metadata(\$c)

* Retrieve metadata for **all** available coverages

>for \$c in /server/coverage return metadata(\$c)

### <a id='xpath_for_where_xwcps'></a><font color="black">Xpath Filtering on For/Where Clause</font>

* Get metadata of all coverages **on endpoint "https://rsg.pml.ac.uk/rasdaman/ows"**

>for \$c in /server[@endpoint='https://rsg.pml.ac.uk/rasdaman/ows']/coverage return metadata(\$c)

* Retrieve metadata for all coverages **on endpoint "http://access.planetserver.eu:8080/rasdaman/ows"** that have **gml:cat_solar_longitude attribute less than 86.0122**

>for \$c in /server[@endpoint='http://access.planetserver.eu:8080/rasdaman/ows']/coverage where metadata(\$c)//gml:cat_solar_longitude[text()<86.0122] return metadata(\$c)

### <a id='xpath_return_xwcps'></a><font color="black">xWCPS request with Xpath on Return Clause</font>

* Get value of **gml:cat_solar_longitude** for all coverages **on endpoint "http://access.planetserver.eu:8080/rasdaman/ows"** that have **gml:cat_solar_longitude attribute less than 86.0122**

>for \$c in /server[@endpoint='http://access.planetserver.eu:8080/rasdaman/ows']/coverage <br /> where metadata(\$c)//gml:cat_solar_longitude[text()<86.0122] return metadata(\$c)//gml:cat_solar_longitude

### <a id='xpath_mixed_result'></a><font color="black">xWCPS request with Mixed Result</font>

* Get a mixed value containing the image of coverage with id "frt0000cc22_07_if165l_trr3" and all its metadata

>for data in (frt0000cc22_07_if165l_trr3) return mixed(encode( { red: (int)(255 / (max((data.band_233 != 65535) \* data.band_233) - min(data.band_233))) \* (data.band_233 - min(data.band_233)); green: (int)(255 / (max((data.band_13 != 65535) \* data.band_13) - min(data.band_13))) \* (data.band_13 - min(data.band_13)); blue: (int)(255 / (max((data.band_78 != 65535) \* data.band_78) - min(data.band_78))) \* (data.band_78 - min(data.band_78)) ; alpha: (data.band_100 != 65535) \* 255}, "png", "nodata=null"), metadata(data))

***

&copy; 2016 ECMWF

This software is licensed under the terms of the Apache Licence Version 2.0 which can be obtained at http://www.apache.org/licenses/LICENSE-2.0. In applying this licence, ECMWF does not waive the privileges and immunities granted to it by virtue of its status as an intergovernmental organisation nor
does it submit to any jurisdiction.